# TM470 Project - Automating the Identification of UK Coarse Fish

In [ ]:
import tensorflow as tf
import kaggle
import pandas as pd
import os
import numpy as np
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et # https://docs.python.org/3/library/xml.etree.elementtree.html
from tensorflow.python.client import device_lib #for detection of devices
from tensorflow.keras import Sequential, optimizers, metrics, layers
# for model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import glob as glob

In [ ]:
# TensorFlow version
print(tf.__version__)

### 3 Is TF using GPU acceleration from inside python shell.

In [ ]:
# Is TF using GPU?
if tf.test.gpu_device_name():
    print('Default GPU device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
# Number of GPU's available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# Details of CPU and GPU from the device library (device_lib)
print(device_lib.list_local_devices())

### AFFiNe dataset from Kaggle (list, download and unzip)

In [ ]:
# List AFFiNe dataset from Kaggle datasets
!kaggle datasets list -s jorritvenema/AFFiNe
# List files in the AFFiNe dataset through Kaggle api
!kaggle datasets files jorritvenema/AFFiNe

In [ ]:
# Download and unzip dataset folder (only run once)
#!kaggle datasets download jorritvenema/AFFiNe --unzip

# Get class names and bound box information from XML files using the parser

In [ ]:
# Dataset address is C:\\Users\\Rob\\Dataset
datasetPath = (r'C:\\Users\\Rob\\Dataset')

In [ ]:
# Assigning dataset path to pathlib
dat_dir = pathlib.Path(datasetPath).with_suffix('')
print(dat_dir)

In [ ]:
# Number of images in dataset
image_count = len(list(dat_dir.glob('*/*.jpg'))) # is this how datasetPath should be?
print(image_count)

In [ ]:
# Reading the information in the XML files and extracting names/bounding box info
path = (dat_dir)
filelist = []
list1 = list()
list2 = list()
for root, dirs, files in os.walk(path):
    for file in files:
        if not file.endswith('.xml'):
            continue
        filelist.append(os.path.join(root, file))
for file in filelist:
    root = et.parse(file).getroot() # get the root of the xml
# Get class names
    for className in root.findall('.//object'):
       class_name = className.find('name').text
       data = np.array([class_name])
       list1.append(data)
# Get bounding box information
    for bndBox in root.findall('.//object'):
       bounding_box = bndBox.find('bndbox').text
       xmin = int(bndBox.find('./bndbox/xmin').text)
       ymin = int(bndBox.find('./bndbox/ymin').text)
       xmax = int(bndBox.find('./bndbox/xmax').text)
       ymax = int(bndBox.find('./bndbox/ymax').text)
       data2 = np.array([xmin,ymin,xmax,ymax])
       list2.append(data2)  

In [ ]:
# print class names and bounding box info lists (list1 and list2)
for i in range (5):
    print(list1[i],list2[i])

## Create dataframe (using relative paths, class names and bound box details from XML)

In [ ]:
#list(base_dir.glob('*/*.jpg'))
filepaths = list(dat_dir.glob(r'**/*.jpg'))
classnames = list1#list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
boundboxes = list2

filepaths = pd.Series(filepaths, name='Filepath').astype(str)#str
classnames = pd.Series(classnames, name='Class Name')
boundboxes = pd.Series(boundboxes, name='Boundbox')

dataframe1 = pd.concat([filepaths , classnames, boundboxes] , axis=1)
dataframe1

In [ ]:
# Useful information on Kaggle:
# https://www.kaggle.com/code/reighns/augmentations-data-cleaning-and-bounding-boxes (3 May 23)
# Hiding id behing jpg
#dataframe1["Filepath"] = dataframe1["Filepath"].apply(lambda x: str(x) + ".jpg")
#dataframe1

# Where to find the test data alternative way


In [ ]:
# my code
# Dataset address is C:\Users\Rob\Dataset
# datasetPath = (r'C:\Users\Rob\Dataset')
# dat_dir = pathlib.Path(datasetPath).with_suffix('')

# A way to split the data
# train_dir = os.path.join(dat_dir, 'train')
# validation_dir = os.path.join(dat_dir, 'validation')
# test_dir = os.path.join(dat_dir, 'test')

In [ ]:
# Assigning dataset path to pathlib
print(dat_dir)

In [ ]:
# Number of images in dataset and dataframe1
image_count = len(list(dat_dir.glob('*/*.jpg')))
image_count_df = len(dataframe1)
print(image_count)
print(image_count_df)

In [ ]:
# The above count for the dataframe is +1, not sure why

# Creating the datasets (how to use dataframe1 created above?)

In [ ]:
# Image size
batch_size=16
img_height=180
img_width=180
image_size=(img_height,img_width,3)
num_classes = 30

In [ ]:
# Create the training dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
  dat_dir,
  validation_split=0.2,
  subset="training",
  seed=123,#none
  image_size=(img_height,img_width),
  batch_size=batch_size)

In [ ]:
# Create the validation dataset
val_dataset = tf.keras.utils.image_dataset_from_directory(
  dat_dir,
  validation_split=0.2,
  subset="validation",
  seed=124,
  image_size=(img_height,img_width),
  batch_size=batch_size)

In [ ]:
# Creating test dataset
test_dataset = tf.keras.utils.image_dataset_from_directory(
  dat_dir,
  #validation_split=0.1,
  #subset="testing",
  seed=125,
  image_size=(img_height,img_width),
  batch_size=batch_size)

In [ ]:
# Assign the class names
class_names = test_dataset.class_names
#class_names=list1
print(class_names) 

In [ ]:
# Next two cells for testing
sample_imgs, sample_labels = test_dataset.as_numpy_iterator().next()
sample_imgs.shape, sample_labels.shape

In [ ]:
# testing using sample label - to try debug final evaluation
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(sample_imgs[i].astype("uint8")) #images[i].numpy().astype("uint8"))
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title(class_names[sample_labels[i]])
plt.show()

## My model (based on TM358 EMA model)

### Data augmentation

In [ ]:
# Creating the normalisation layer
norm_layer = layers.Normalization(input_shape=(image_size))
norm_layer.adapt(train_dataset.map(lambda x, y: x))

In [ ]:
# Creating an augmented subset
data_augmentation = tf.keras.Sequential([
layers.RandomRotation(0.2),
#layers.RandomZoom(height_factor=0.1),
layers.RandomFlip(mode='horizontal')
])

aug_train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
num_parallel_calls=tf.data.AUTOTUNE)
aug_train_dataset = aug_train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

### Model creation

In [ ]:
# Define function to create model and compile
def build_model():
    model = Sequential([
        #norm_layer, # to normalise data - making training freeze
        Conv2D(filters=16, kernel_size=(3,3), padding='same',
        input_shape=image_size, activation='relu'),
        Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'),
        Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'),
        Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'),
        Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'),
        Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'),
        Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'),
        Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'),
        Conv2D(filters=64, kernel_size=(3,3), padding='valid', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.5),
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',#(learning_rate=0.005),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
        )
    return model

In [ ]:
# Build the model using the build_model function
model=build_model()

In [ ]:
# Show a summary of the model
model.summary()

### Train the model

In [ ]:
# Train the model
#with tf.device("/device:GPU:0"):
hist=model.fit(
aug_train_dataset, 
validation_data=val_dataset, 
verbose=1,
epochs=5)

In [ ]:
# Plotting training loss and accuracy as well as validation loss and accuracy over the number of epochs
hist_dict = hist.history

# obtain the accuracy and loss of the training set and verification set in the returned
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(1, len(train_acc)+1)
plt.plot(epochs, train_acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'r', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend() # show legend 
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()
plt.figure()

plt.plot(epochs, train_loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')

### Evaluating the model (based on code from TM358)

In [ ]:
model.evaluate(test_dataset, return_dict=True)

In [ ]:
test_predictions=model.predict(test_dataset)
test_predictions.shape

In [ ]:
predict_labels = np.argmax(test_predictions, axis=1)
predict_labels.shape

In [ ]:
actual_labels=np.array(list(test_dataset.unbatch().map(lambda x,y: y).as_numpy_iterator()))
#actual_labels=class_names
actual_labels=np.argmax(actual_labels, axis=0)
actual_labels.shape
actual_labels
#print(actual_labels)

In [ ]:
#  actual_label = LABEL_TEXT[test_labels[i]]

In [ ]:
sample_imgs, sample_labels = test_dataset.as_numpy_iterator().next()
sample_imgs.shape, sample_labels.shape, sample_labels

In [ ]:
sample_predictions = model(sample_imgs)
# View the true and predicted labels of sample images
plt.figure(figsize=(15,15))
for i in range(15):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(sample_imgs[i].astype("uint8"))
    #plt.imshow(sample_imgs[i])
    p_class = np.argmax(sample_predictions[i])
    a_class = np.argmax(sample_labels[i])#sample
    #plt.title(f"P: {class_names[p_class]}\n(A: {class_names[a_class]})",
    plt.title(f"P: {class_names[p_class]}\n(A: {class_names[a_class]})",# class_names[a_class]
    color=("green" if p_class == a_class else "red"))
    plt.axis("off")
plt.show()